#### 获得初始电影实体

In [4]:
# 读取 id 列表
import pickle

origin_movies = set()
def entry0():

    # 构建实体-tag字典，同时匹配获得Freebase中对应的实体（共578个可匹配实体），加入到一跳可匹配实体
    with open('douban2fb.txt', 'rb') as f:
        for line in f:
            line = line.strip()
            parts = line.decode().split('\t')
            origin_movies.add(parts[1])
    print("entry0:", len(origin_movies))
    with open("entry0.pkl", "wb") as f:
        pickle.dump(origin_movies, f)


entry0()


entry0: 578


#### 一跳得到子图并对实体和关系计数

In [22]:
import gzip
import pickle
from tqdm import tqdm

template_str = "http://rdf.freebase.com/ns/m."
template_str2 = "http://rdf.freebase.com/ns/"
freebase_info_fpath = "freebase_douban.gz"  # 初始freebase
outfile1 = "graph_1step.gz"  # 一跳输出文件
outfile2 = "graph_2step.gz"  # 二跳输出文件
outfile3 = "graph_3step.gz"

entry_num = {}
def step1():
    # 以 mvi_entities 为起点生成一跳子图，保存到 grarph1step.gz 文件中
    with gzip.open(outfile1, 'wb') as ans:
        with open('entry0.pkl', 'rb') as f1:
            mvi_entities = pickle.load(f1)
        with gzip.open(freebase_info_fpath, 'rb') as f:

            for line in tqdm(f, total=395577070):
                # if count > max_run_times:
                #     break  # 超过设定上限即停止查找一跳可达实体
                line = line.strip()
                triple_parts = line.decode().split('\t')
                if (template_str not in triple_parts[0]
                        or template_str2 not in triple_parts[1]
                        or template_str not in triple_parts[2]):
                    continue
                # 头实体
                head = triple_parts[0][len(template_str2) + 1:].strip('>')
                
                # 关系
                rel = triple_parts[1][len(template_str2) + 1:].strip('>')
                # 尾实体
                tail = triple_parts[2][len(template_str2) + 1:].strip('>')

                # 保存头实体在 mvi_entities 中的三元组
                if head in mvi_entities:
                    ans.write(f'{head}\t{rel}\t{tail}\n'.encode('utf-8'))
                    # 实体相关三元组计数
                    if (head in entry_num):
                        entry_num[head] += 1
                    else:
                        entry_num[head] = 1
                    if (tail in entry_num):
                        entry_num[tail] += 1
                    else:
                        entry_num[tail] = 1
                    if (rel in entry_num):
                        entry_num[rel] += 1
                    else:
                        entry_num[rel] = 1


step1()
# print(entry_num)

100%|██████████| 395577070/395577070 [07:42<00:00, 854799.65it/s] 

{'m.012x63': 116, 'm.03k9fj': 151, 'film.film.genre': 2646, 'm.0p7zw8x': 1, 'film.film.dubbing_performances': 450, 'm.0zcrwbm': 1, 'film.film.distributors': 1086, 'm.0p7zxhh': 1, 'm.0534v': 27, 'film.film.directed_by': 675, 'm.02kdv5l': 178, 'm.0jsg5r': 90, 'freebase.valuenotation.is_reviewed': 1252, 'm.0ngwjn0': 1, 'film.film.starring': 23671, 'm.0ngwjjz': 1, 'm.0282qln': 11, 'film.film.edited_by': 781, 'm.0gw5n2f': 27, 'media_common.netflix_title.netflix_genres': 4908, 'm.0g2f_3h': 1, 'common.topic.image': 576, 'm.0wzmrh1': 1, 'base.schemastaging.context_name.pronunciation': 76, 'm.0jw5bd': 1, 'film.film.runtime': 1352, 'm.0jsg5z': 54, 'm.0cq238g': 17, 'm.0cq23v1': 8, 'm.0jsg5j': 54, 'm.0gw5ykn': 11, 'm.0jsg59': 249, 'm.0h5rq35': 1, 'film.film.cinematography': 610, 'm.0nfzmk4': 1, 'film.film.release_date_s': 12495, 'm.04d5yl8': 1, 'film.film.soundtrack': 502, 'm.0j56b_y': 1, 'm.0ngwjsr': 1, 'm.0gw5ndc': 58, 'm.0p7zz00': 1, 'm.0jsg2m': 310, 'common.topic.notable_types': 578, 'm.02h40l

#### 筛选一跳子图

In [39]:
def Select1():
    with gzip.open(outfile1, 'rb') as f:
        mvi_entities2 = set()
        with open("entry0.pkl", "rb") as f1:
            mvi_entities = pickle.load(f1)
        count = 0
        for line in tqdm(f, total=110759):
            triple_parts = line.decode().split('\t')[:3]
            # 头实体
            head = triple_parts[0]
            # 关系
            rel = triple_parts[1]
            # 尾实体
            tail = triple_parts[2][:-1] # 删除末尾的\n
            # print(head,rel,tail)
            if ((head in mvi_entities or entry_num[head] > 20)
                    and (entry_num[tail] > 20)
                    and (rel in mvi_entities or entry_num[rel] > 50)):
                mvi_entities2.add(head)
                mvi_entities2.add(tail)
            count += 1
        print("1step:", count)
        print("entry1:", len(mvi_entities2))
        with open("entry1.pkl", "wb") as f:
            pickle.dump(mvi_entities2, f)
        # print(mvi_entities2)


Select1()

100%|██████████| 110759/110759 [00:00<00:00, 802196.69it/s]

1step: 110759
entry1: 728


#### 二跳子图

In [28]:
entry_num2 = {}  # 二跳元素存储


def step2():
    with gzip.open(outfile2, 'wb') as ans:
        with open('entry1.pkl', 'rb') as f1:
            mvi_entities = pickle.load(f1)
        with gzip.open(freebase_info_fpath, 'rb') as f:
            for line in tqdm(f, total=395577070):
                # if count > max_run_times:
                #     break  # 超过设定上限即停止查找一跳可达实体
                line = line.strip()
                triple_parts = line.decode().split('\t')
                if (template_str not in triple_parts[0]
                        or template_str2 not in triple_parts[1]
                        or template_str not in triple_parts[2]):
                    continue
                # 头实体
                head = triple_parts[0][len(template_str2) + 1:].strip('>')
                # 关系
                rel = triple_parts[1][len(template_str2) + 1:].strip('>')
                # 尾实体
                tail = triple_parts[2][len(template_str2) + 1:].strip('>')

                # 保存头实体在 mvi_entities 中的三元组
                if head in mvi_entities:
                    ans.write(f'{head}\t{rel}\t{tail}\n'.encode('utf-8'))
                    # 实体相关三元组计数
                    if (head in entry_num2):
                        entry_num2[head] += 1
                    else:
                        entry_num2[head] = 1
                    if (tail in entry_num2):
                        entry_num2[tail] += 1
                    else:
                        entry_num2[tail] = 1
                    if (rel in entry_num2):
                        entry_num2[rel] += 1
                    else:
                        entry_num2[rel] = 1


step2()

100%|██████████| 395577070/395577070 [07:51<00:00, 839261.98it/s] 


#### 第一次筛选二跳子图

In [41]:
def Select2():
    count = 0
    with gzip.open(outfile2, 'rb') as f:
        mvi_entities2 = set()
        with open("entry1.pkl", "rb") as f1:
            mvi_entities = pickle.load(f1)
        for line in tqdm(f, total=1119177):
            triple_parts = line.decode().split('\t')[:3]
            # 头实体
            head = triple_parts[0]
            # 关系
            rel = triple_parts[1]
            # 尾实体
            tail = triple_parts[2][:-1]
            if ((head in mvi_entities or entry_num2[head] <= 20000)
                    and (entry_num2[tail] <= 20000)
                    and (rel in mvi_entities or entry_num2[rel] > 50)):
                mvi_entities2.add(head)
                mvi_entities2.add(tail)
                count += 1
        print('step2:',count)
        print("entry2:", len(mvi_entities2))
        with open("entry2.pkl", "wb") as f:
            pickle.dump(mvi_entities2, f)


Select2()

100%|██████████| 1119177/1119177 [00:01<00:00, 610644.76it/s]


step2: 1113785
entry2: 673442


#### 对筛选过的二跳子图重新计数

In [31]:
entry_num3 = {}  # 二跳首次筛选后元素存储
def step3():
    with gzip.open(outfile3, 'wb') as ans:
        with open('entry2.pkl', 'rb') as f1:
            mvi_entities = pickle.load(f1)
        with gzip.open(outfile2, 'rb') as f:
            for line in tqdm(f, total=1119177):
                # if count > max_run_times:
                #     break  # 超过设定上限即停止查找一跳可达实体
                line = line.strip()
                triple_parts = line.decode().split('\t')
                # if (template_str not in triple_parts[0]
                #         or template_str not in triple_parts[1]
                #         or template_str not in triple_parts[2]):
                #     continue
                # 头实体
                head = triple_parts[0]
                # 关系
                rel = triple_parts[1]
                # 尾实体
                tail = triple_parts[2][:-1]

                # 保存头实体在 mvi_entities 中的三元组
                if head in mvi_entities:
                    ans.write(line + b'\n')


step3()

  1%|          | 1119177/104715852 [00:05<08:27, 204191.55it/s]


#### 迭代筛选

In [32]:
def Select(triplets):
    entry_num = {}
    for triplet in triplets:
        if (triplet[0] not in entry_num):
            entry_num[triplet[0]] = 0
        entry_num[triplet[0]] += 1
        if (triplet[2] not in entry_num):
            entry_num[triplet[2]] = 0
        entry_num[triplet[2]] += 1
        if (triplet[1] not in entry_num):
            entry_num[triplet[1]] = 0
        entry_num[triplet[1]] += 1
    ans = []
    for triplet in triplets:
        if ((triplets[0] in origin_movies or entry_num[triplet[0]]>15)and(triplets[2] in origin_movies or entry_num[triplet[2]]>15)and(entry_num[triplet[1]]>50)):
            ans.append(triplet)
    return ans

#### 二次筛选二跳子图

In [37]:
import gzip
import pickle
from tqdm import tqdm


freebase_info_fpath = "freebase_douban.gz"  # 初始freebase
outfile1 = "graph_1step.gz"  # 一跳输出文件
outfile2 = "graph_2step.gz"  # 二跳输出文件
outfile3 = "graph_3step.gz"


def Select3():
    before_selected = []
    with gzip.open(outfile3, 'rb') as f:
        for line in tqdm(f, total=1119177):
            triple_parts = line.decode().split('\t')[:3]
            # 头实体
            head = triple_parts[0]
            # 关系
            rel = triple_parts[1]
            # 尾实体
            tail = triple_parts[2][:-1]
            before_selected.append((head, rel, tail))
        print('get before_selected')
        selected = Select(before_selected)
        print('筛选后:', len(selected), '筛选前:', len(before_selected))

    while len(selected) < len(before_selected):  # 迭代筛选
        before_selected = selected
        selected = Select(before_selected)
        print('筛选后:', len(selected), '筛选前:', len(before_selected))

    with gzip.open('final.gz', 'wb') as ans:
        count = 0
        mvi_entities2 = set()
        for triplet in selected:
            mvi_entities2.add(triplet[0])
            mvi_entities2.add(triplet[2])
            ans.write(f'{triplet[0]}\t{triplet[1]}\t{triplet[2]}\n'.encode('utf-8'))
        print("final:", len(selected))
        print("entry3:", len(mvi_entities2))
        with open("entry3.pkl", "wb") as f:
            pickle.dump(mvi_entities2, f)


Select3()

100%|██████████| 1119177/1119177 [00:01<00:00, 940612.98it/s]


get before_selected
筛选后: 48528 筛选前: 1119177
筛选后: 47690 筛选前: 48528
筛选后: 47282 筛选前: 47690
筛选后: 47164 筛选前: 47282
筛选后: 47022 筛选前: 47164
筛选后: 47006 筛选前: 47022
筛选后: 47006 筛选前: 47006
final: 47006
entry3: 2319
